<a href="https://colab.research.google.com/github/sharathpandurajbaliga/deeplearning.FaceDetection/blob/main/FaceDetection_IBUG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connect to the google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Check if GPU available

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Jul 16 01:00:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

All imports

In [3]:
import torch
import torch.nn as nn
import numpy as np
import json
import glob
import cv2
import os
import ast
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
import pandas as pd
import csv
from random import seed
from random import randint
seed(1)
import math
import shutil
from torch.utils.data.sampler import SubsetRandomSampler
import torch.optim as optim
import xml.etree.ElementTree as ET

Dataset Preparation

In [4]:
class dataset_util(Dataset):
  def __init__(self, data_basedir, traindir, label_path, transform):
    self.data_basedir = data_basedir
    self.traindir = traindir
    self.label_path = label_path
    self.transform = transform
    self.label_ids = self.__iterate_through_labels()

  def __iterate_through_images(self, dirs):
    image_ids = []
    for dir in dirs:
      image_path = os.path.join(self.data_basedir, dir)
      for img in glob.glob(image_path + "/*.jpg"):
          image_ids.append(img)
    return image_ids

  def __iterate_through_labels(self):
    label_ids = {}
    tree = ET.parse(os.path.join(self.data_basedir, self.label_path))
    root = tree.getroot()
    count = 0
    for image in root.find('images'):
      #boxElement = box.getElementsByTagName('box')[0]
      #print(image.attrib)
      box = image.find('box')
      image_width = int(image.attrib['width'])
      image_height = int(image.attrib['height'])
      scale_w = 400/image_width
      scale_h = 400/image_height
      top = int(int(box.attrib['top'])*scale_h)
      left = int(int(box.attrib['left'])*scale_w)
      width = int(int(box.attrib['width'])*scale_w)
      height = int(int(box.attrib['height'])*scale_h)
      label_ids[count] = [image.attrib['file'], top, left, width, height]
      count = count + 1
    return label_ids


  def __getitem__(self, idx):
    label_list = self.label_ids[idx]
    img = self.__load_image(os.path.join(self.data_basedir, label_list[0]))
    tensor_image = self.transform(img)
    label = self.__get_label(label_list)
    return tensor_image, label

  def __len__(self):
    return len(self.label_ids.keys())
  
  def __load_image(self, image_path):
    image_path = os.path.join(self.data_basedir, image_path)
    img = cv2.imread(image_path)
    img = cv2.resize(img, (400,400)) # INTER_AREA
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img
  
  def __get_label(self, label_list):
    return np.array(label_list[1:])



Face Detection 

In [5]:
class face_detection:
  def __init__(self, train_loader, valid_loader, model, criterion, optimizer, epochs, model_to_save):
    self.criterion = criterion
    self.optimizer = optimizer
    self.model = model
    self.epochs = epochs
    self.train_on_gpu = torch.cuda.is_available()
    self.model_to_save = model_to_save
    if self.train_on_gpu:
      self.model.cuda()

    self.train_loader = train_loader
    self.valid_loader = valid_loader
    return 

  def train(self):
    # track change in validation loss
    valid_loss_min = np.Inf 
    print("Starting the train...")
    for epoch in range(1, self.epochs+1):

        # keep track of training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        self.model.train()
        for data, target in train_loader:
            # move tensors to GPU if CUDA is available
            if self.train_on_gpu:
                data, target = data.cuda(), target.cuda()
            # clear the gradients of all optimized variables
            self.optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = self.model(data)
            # calculate the batch loss
            loss = self.criterion(output, target.float())
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            self.optimizer.step()
            # update training loss
            train_loss += loss.item()*data.size(0)
        ######################    
        # validate the model #
        ######################
        count = 0
        self.model.eval()
        for data, target in valid_loader:
            # move tensors to GPU if CUDA is available
            if self.train_on_gpu:
                data, target = data.cuda(), target.cuda()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = self.model(data)
            # calculate the batch loss
            loss = self.criterion(output, target.float())
            # update average validation loss 
            valid_loss += loss.item()*data.size(0)
        
        # calculate average losses
        train_loss = train_loss/len(train_loader.dataset)
        valid_loss = valid_loss/len(valid_loader.dataset)
            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, train_loss, valid_loss))
        
        # save model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(self.model.state_dict(), self.model_to_save)
            valid_loss_min = valid_loss
    return

  def test(self, model_path):
    self.model.load_state_dict(torch.load(model_path))
    # track test loss
    test_loss = 0.0
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))

    self.model.eval()
    # iterate over test data
    for data, target in test_loader:
        # move tensors to GPU if CUDA is available

        if self.train_on_gpu:
            data, target = data.cuda(), target.cuda()

        # forward pass: compute predicted outputs by passing inputs to the model
        output = self.model(data)

        # calculate the batch loss
        loss = self.criterion(output, target)

        # update test loss 
        test_loss += loss.item()*data.size(0)

    # average test loss
    test_loss = test_loss/len(test_loader.dataset)
    print('Test Loss: {:.6f}\n'.format(test_loss))
    return

  

Models

In [6]:
class myResNetModel(nn.Module):
  def __init__(self):
    super(myResNetModel, self).__init__()
    self.model = models.resnet18()
    self.model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
    self.model.fc = nn.Linear(self.model.fc.in_features, 4)
    return 

  def forward(self, x):
    x = self.model(x)
    return x

Code starts running from here

In [7]:
data_transform = transforms.Compose([transforms.ToTensor()])

base_dir = "/content/drive/MyDrive/MyProjects/300W_FaceProject/ibug_300W_large_face_landmark_dataset"
train_dir = ["afw","helen", "ibug", "lfpw"]
train_label = "labels_ibug_300W_train.xml"
test_label = "labels_ibug_300W_test.xml"

train_data = dataset_util(base_dir, train_dir, train_label, data_transform)
test_data = dataset_util(base_dir, train_dir, test_label, data_transform)

# print out some data stats
train_landmarks_length = len(train_data)
print('Total Number of Images: ', train_landmarks_length )


batch_size = 64
num_workers = 0

num_train = len(train_data)
num_test = len(test_data)
train_indices = list(range(num_train))
test_indices = list(range(num_test))
np.random.shuffle(train_indices)

train_split = int(np.floor(0.85 * num_train))
valid_split = int(num_train - train_split)

train_idx = train_indices[:train_split]
valid_idx = train_indices[train_split:]
test_idx = test_indices


# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)
test_sampler = SubsetRandomSampler(test_idx)

# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,sampler=test_sampler, num_workers=num_workers)

model = myResNetModel()
epochs = 100

# specify loss function
criterion = nn.MSELoss()

# specify optimizer
optimizer = torch.optim.Adam(model.parameters())

face_detection_util = face_detection(train_loader, valid_loader, model, criterion, optimizer, epochs, os.path.join(base_dir,"face_detector_gray_400_400.pt"))




Total Number of Images:  6666


In [ ]:
face_detection_util.train()

Starting the train...


In [ ]:
face_detection_util.test(os.path.join(base_dir,"face_detector_gray_400_400.pt"))

Test Loss: 12.835593

